In [ ]:
!pip3 install agentpy

In [48]:
import agentpy as ap, numpy as np
from enum import Enum
from matplotlib import pyplot as plt
from IPython.display import HTML
from queue import PriorityQueue

In [104]:
# Set up enum and functions

class AgentType(Enum):
    """Enum for each type of agent"""
    RANDOM = "random",
    LARGEST = "largest_reward",
    CLOSEST = "closest_reward"

def my_plot(model, ax):
    """Function for plotting"""
    grid = np.zeros(model.environment.shape)
    print(model.environment.positions)
    for agent, pos in model.environment.positions.items():
        grid[pos] = agent.id
    ax.imshow(grid, cmap='Greys')

In [100]:
# Model class to test interactions
class SimpleModel(ap.Model):

    def __init__(self, steps: int, type: AgentType, energy: int, initial_position: float, grid):
        parameters = {'print': False, 'steps':steps}
        super().__init__(parameters=parameters)
        self.grid = grid
        self.environment = ap.Grid(self, (len(self.grid), len(self.grid[0])))
        self.agent = ap.Agent(self)
        self.environment.add_agents([self.agent], positions=[initial_position])
        self.type = type
        self.target = None
        self.energy = energy
        self.setup_rewards()

    def step(self):
        if self.energy <= 0:
            print("No energy")
            return
        
        if self.type is AgentType.RANDOM:
            self.step_random()
        elif self.type is AgentType.LARGEST:
            self.step_largest()
        elif self.type is AgentType.CLOSEST:
            self.step_closest()
        
        self.energy -= 1

    def setup_rewards(self):
        if self.type is AgentType.RANDOM:
            self.moves = [(0,1), (0,-1), (1,0), (-1,0)]

        elif self.type is AgentType.LARGEST:
            self.pq = PriorityQueue()
            
            for i in range(len(self.grid)): 
                for j in range(len(self.grid[i])):  
                    value = self.grid[i][j]
                    
                    if value != 0:  
                        self.pq.put((value, (i, j))) 
        
        elif self.type is AgentType.CLOSEST:
           self.rewards = []

           for i in range(len(self.grid)): 
                for j in range(len(self.grid[i])):  
                    value = self.grid[i][j]
                    
                    if value != 0:  
                        self.rewards.append((value, (i,j)))

    def move(self) -> None:
        move = (0,0)

        if self.target[0] > self.current_position[0]:
            move = (1,0)
        elif self.target[0] < self.current_position[0]:
            move = (-1,0)
        elif self.target[1] > self.current_position[1]:
            move = (0,1)
        elif self.target[1] < self.current_position[1]:
            move = (0,-1)

        self.environment.move_by(self.agent, move)

    def add_value(self):
        if self.grid[self.current_position[0]][self.current_position[1]] != 0:
                self.energy += self.grid[self.current_position[0]][self.current_position[1]] 
                self.grid[self.current_position[0]][self.current_position[1]] = 0

    def step_random(self):
        move = self.moves[np.random.randint(0,4)]
        self.environment.move_by(self.agent, move)
    
    def step_largest(self):
        self.current_position = self.environment.positions[self.agent]

        if self.target is None or self.target == self.current_position:
            self.add_value()
            
            if not self.pq.empty():
                _, self.target = self.pq.get()
            else:
                _, self.target = 0, self.current_position
                
        self.move()
    
    def step_closest(self):
        self.current_position = self.environment.positions[self.agent] 
        
        if self.target is None or self.target == self.current_position:
            self.add_value()

            closest_reward = (np.inf, self.current_position)

            for reward in self.rewards:
                distance = abs(self.reward[0] - self.current_position[0]) + abs(self.reward[1] - self.current_position[1])

                if distance < closest_reward[0]:
                    closest_reward = (distance, reward)

            self.target = closest_reward[1]
            self.rewards.remove(closest_reward[1])

        self.move()




In [105]:
fig, ax = plt.subplots()
ENERGY = 10
INITIAL_POSITION = (0,0)
grid = [[0.0, 0.0, -1.0, 0.0, 2.0], [0.0, 0.0, 0.0, 3.0, 0.0], [-3.0, 0.0, 1.0, 0.0, 0.0], [3.0, 1.0, 0.0, 0.0, 4.0], [0.0, 0.0, 0.0, -4.0, 4.0]]
simpleModel = SimpleModel(10,AgentType.LARGEST,10, (0,0), grid)
animation = ap.animate(simpleModel, fig, ax, my_plot)
HTML(animation.to_jshtml())

{Agent (Obj 2): (0, 0)}
{Agent (Obj 2): (0, 0)}
{Agent (Obj 2): (1, 0)}
{Agent (Obj 2): (2, 0)}
{Agent (Obj 2): (3, 0)}
{Agent (Obj 2): (4, 0)}
{Agent (Obj 2): (4, 1)}
{Agent (Obj 2): (4, 2)}
{Agent (Obj 2): (4, 3)}
{Agent (Obj 2): (3, 3)}
No energy
{Agent (Obj 2): (3, 3)}
No energy
{Agent (Obj 2): (3, 3)}
